Теперь решаем задачу регрессии - предскажем цены на недвижимость. Использовать датасет https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data (train.csv)
Данных немного, поэтому необходимо использовать 10-fold кросс-валидацию для оценки качества моделей
Построить случайный лес, вывести важность признаков
Обучить стекинг как минимум 3х моделей, использовать хотя бы 1 линейную модель и 1 нелинейную
Для валидации модели 2-го уровня использовать отдельный hold-out датасет, как на занятии
Показать, что использование ансамблей моделей действительно улучшает качество (стекинг vs другие модели сравнивать на hold-out)
В качестве решения: Jupyter notebook с кодом, комментариями и графиками, ссылка на гитхаб

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.metrics import auc, roc_curve, roc_auc_score, f1_score, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, RandomForestRegressor, GradientBoostingRegressor
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier, XGBRegressor
from sklearn.preprocessing import StandardScaler
%matplotlib inline

In [ ]:
df = pd.read_csv("train_new.csv")

In [ ]:
df.head()

In [ ]:
Y = df['SalePrice'].copy()
X = df.drop('SalePrice', axis=1)

In [5]:
X.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal


In [6]:
X.isnull().sum()[X.isnull().sum().values != 0]

LotFrontage      259
Alley           1369
MasVnrType         8
MasVnrArea         8
BsmtQual          37
BsmtCond          37
BsmtExposure      38
BsmtFinType1      37
BsmtFinType2      38
Electrical         1
FireplaceQu      690
GarageType        81
GarageYrBlt       81
GarageFinish      81
GarageQual        81
GarageCond        81
PoolQC          1453
Fence           1179
MiscFeature     1406
dtype: int64

In [7]:
cat_feature = list(df.dtypes[df.dtypes == object].index)

In [8]:
df_cat = df[cat_feature]
df_int = df.drop(cat_feature, axis=1)

In [9]:
df[df_cat.columns] = df_cat[df_cat.columns].fillna('NaN')
df[df_int.columns] = df_int[df_int.columns].fillna(0)

In [10]:
df.isnull().sum()[df.isnull().sum().values != 0]
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [11]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
df = df.select_dtypes(include=numerics).fillna(0)

In [12]:
df.head()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
0,1,60,65.0,8450,7,5,2003,2003,196.0,706,...,0,61,0,0,0,0,0,2,2008,208500
1,2,20,80.0,9600,6,8,1976,1976,0.0,978,...,298,0,0,0,0,0,0,5,2007,181500
2,3,60,68.0,11250,7,5,2001,2002,162.0,486,...,0,42,0,0,0,0,0,9,2008,223500
3,4,70,60.0,9550,7,5,1915,1970,0.0,216,...,0,35,272,0,0,0,0,2,2006,140000
4,5,60,84.0,14260,8,5,2000,2000,350.0,655,...,192,84,0,0,0,0,0,12,2008,250000


In [13]:
Y = df['SalePrice'].copy()
X = df.drop('SalePrice', axis=1)

In [14]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn import svm
from xgboost import XGBClassifier, XGBRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, shuffle=True)

# делаем нормирование
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [41]:
clf_rf = RandomForestRegressor()
n_estimators = [200, 400]
max_depth = [5, 9]
min_samples_split = [15, 20]
param_grid = {'max_depth': max_depth, 'n_estimators': n_estimators, 'min_samples_split':min_samples_split}

clf_rf = GridSearchCV(clf_rf, param_grid, cv=10, n_jobs=-1, scoring='neg_mean_squared_error')
clf_rf.fit(X_train, y_train)

C:\Users\korshunova_av\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=RandomForestRegressor(bootstrap=True, criterion='mse',
                                             max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators='warn', n_jobs=None,
                                             oob_score=False, random_state=None,
                                             verbose=0, warm_start=False),
             iid='warn', n_jobs=-1,
             param_grid={'max_depth': [3, 5, 9],
        

In [42]:
clf_rf_best = clf_rf.best_estimator_
clf_rf_best

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=9,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=400,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [43]:
y_pred = clf_rf_best.predict(X_test)

In [44]:
# Оценка по RandomForestRegressor
from sklearn.metrics import mean_squared_error
lin_mse = mean_squared_error(y_test, y_pred)
lin_rmse_rf = np.sqrt(lin_mse)
lin_rmse_rf

32732.214940919002

In [20]:
# Выводим важность признаков (TOP-5) 
importances = list(clf_rf_best.feature_importances_)
feature_importances = [(feature, round(importance, 5)) for feature, importance in zip(list(X.columns), importances)]
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
[print('Variable: | {:20} | Importance: {}'.format(*pair)) for pair in feature_importances];

Variable: | OverallQual          | Importance: 0.62908
Variable: | GrLivArea            | Importance: 0.11085
Variable: | 2ndFlrSF             | Importance: 0.04053
Variable: | TotalBsmtSF          | Importance: 0.03505
Variable: | BsmtFinSF1           | Importance: 0.03043
Variable: | GarageArea           | Importance: 0.02043
Variable: | 1stFlrSF             | Importance: 0.02042
Variable: | FullBath             | Importance: 0.01969
Variable: | LotArea              | Importance: 0.01489
Variable: | YearBuilt            | Importance: 0.00945
Variable: | YearRemodAdd         | Importance: 0.00758
Variable: | GarageCars           | Importance: 0.00719
Variable: | TotRmsAbvGrd         | Importance: 0.00657
Variable: | GarageYrBlt          | Importance: 0.00592
Variable: | OverallCond          | Importance: 0.00458
Variable: | LotFrontage          | Importance: 0.00435
Variable: | OpenPorchSF          | Importance: 0.00413
Variable: | WoodDeckSF           | Importance: 0.00397
Variable: 

In [31]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso


lasso = Lasso(random_state=1)
rf = RandomForestRegressor(n_estimators=400, max_depth=15, min_samples_split=15)
xgb = XGBRegressor(learning_rate=0.01, n_estimators=400, max_depth=15,objective ='reg:squarederror',)
lr_meta = LinearRegression(n_jobs=-1)

In [37]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [38]:
from sklearn.model_selection import StratifiedKFold, cross_val_score
from mlxtend.regressor import StackingRegressor

sclf = StackingRegressor(regressors=[rf, xgb, lasso], 
                          meta_regressor=lr_meta)

print('10-fold cross validation:\n')

for clf, label in zip([rf, xgb, lasso, sclf], 
                      ['Random Forest', 
                       'XGBoost',
                       'Lasso',
                       'StackingRegressor']):

    scores = cross_val_score(clf, X, Y, cv=10)
    print("Accuracy: %0.2f (+/- %0.2f) [%s]" 
          % (scores.mean(), scores.std(), label))

10-fold cross validation:

Accuracy: 0.85 (+/- 0.04) [Random Forest]
Accuracy: 0.85 (+/- 0.04) [XGBoost]
Accuracy: 0.80 (+/- 0.10) [Lasso]
Accuracy: 0.85 (+/- 0.05) [StackingRegressor]


In [39]:
sclf.fit(X_train, y_train)
y_pred = sclf.predict(X_test)

In [40]:
score = mean_squared_error(y_test, y_pred)
score = np.sqrt(score)
score

28189.046192770747